## Goal 

Use time-series forecasting to forecast store sales on data from Corporación Favorita, a large Ecuadorian-based grocery retailer. <br>
<br>
Specifically, build a model that more accurately predicts the unit sales for thousands of items sold at different Favorita stores. You'll practice your machine learning skills with an approachable training dataset of dates, store, and item information, promotions, and unit sales.

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 

The **training data**, comprising time series of features store_nbr, family, and onpromotion as well as the target sales : <br>
<br>
- **store_nbr** identifies the store at which the products are sold <br>
<br>
- **family** identifies the type of product sold <br>
<br>
- **sales** gives the total sales for a product family at a particular store at a given date <br>
Fractional values are possible since products can be sold in fractional units (1.5 kg of cheese, for instance, as opposed to 1 bag of chips). <br>
<br>
- **onpromotion** gives the total number of items in a product family that were being promoted at a store at a given date. <br>
<br>
<br>
Test csv looks like train csv. <br>
Target : predict '**sales**' in the test csv. <br>
The dates in the test data are for the 15 days after the last date in the training data.

In [4]:
df_train = pd.read_csv('/Users/augustincablant/Desktop/kaggle/train.csv')
df_train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [2]:
transaction = pd.read_csv('/Users/augustincablant/Desktop/kaggle/transactions.csv')
transaction.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [ ]:
store = pd.read_csv('datasets/')